In [10]:
#@title Check residues at specific positions (1-based)
#@markdown **راهنما:** یکی از سه روش زیر را استفاده کن:
#@markdown 1) توالی را مستقیم اینجا بچسبان، یا
#@markdown 2) یک فایل FASTA آپلود کن، یا
#@markdown 3) هر دو را خالی بگذار و از توالی نمونه استفاده شود.
from typing import List, Tuple
import re

# --- ورودی‌ها ---
seq_input = "MAVAEAAAAALRAQGLDALADAFLAEARRAEELGRAAGDAQVAVAGSQAGHRGLLAALALGRAADAGRTMLAAALDAARAALARPEGRETAAALLAALERDYAAFAALTPEELRALGERTAAALLAAAVAALAATDPAAYAALLAALAAA"  #@param {type:"string"}
positions = "47,51,101"  #@param {type:"string"}
expected = "S,H,D"       #@param {type:"string"}

use_fasta_upload = False  #@param {type:"boolean"}
fasta_path = ""           #@param {type:"string"}

# اگر خواستی از آپلود دستی استفاده کنی (Colab):
try:
    from google.colab import files  # type: ignore
    if use_fasta_upload and not fasta_path:
        print("➡️ لطفاً فایل FASTA را انتخاب کنید...")
        up = files.upload()
        if up:
            fasta_path = list(up.keys())[0]
except Exception:
    pass

# --- توالی نمونه (اگر ورودی ندادی) ---
sample_seq = "RAAADAAAAALRAAGLDALADRWRAAADAAAAAGDAAGDADVAVAGSSAAHRGLLAALALGRAAAAGAAMLDATIAALEEALKAPEGREEAAAALAAVERDRAAFLALTPAELAALGAATAAALAAAALAALAATDPAAFAAVLAALAAA"

def read_fasta(path: str) -> Tuple[str, str]:
    """FASTA ساده: فقط اولین رکورد خوانده می‌شود."""
    header, seq = "", []
    with open(path) as f:
        for line in f:
            if line.startswith(">"):
                if not header:
                    header = line.strip()
                else:
                    break
            else:
                seq.append(line.strip())
    return header or ">seq", re.sub(r"\s+", "", "".join(seq))

def clean_seq(s: str) -> str:
    s = s.strip().upper().replace(" ", "")
    # حذف کاراکترهای غیر از 20 aa استاندارد + B, Z, X (اگر وجود داشتند)
    s = re.sub(r"[^ACDEFGHIKLMNPQRSTVWYBZX]", "", s)
    return s

# انتخاب توالی
if fasta_path:
    hdr, seq = read_fasta(fasta_path)
    print(f"FASTA header: {hdr}")
    seq = clean_seq(seq)
elif seq_input.strip():
    seq = clean_seq(seq_input)
else:
    print("⚠️ توالی ورودی ندادید؛ از توالی نمونه استفاده شد.")
    seq = sample_seq

# پردازش پوزیشن‌ها و انتظارات
pos_list: List[int] = []
for tok in positions.split(","):
    tok = tok.strip()
    if tok.isdigit():
        pos_list.append(int(tok))

exp_list = [e.strip().upper() for e in expected.split(",")] if expected.strip() else []

# تابع گرفتن حرف از پوزیشن ۱-بیسی
def aa_at(seq: str, pos: int) -> str:
    return seq[pos-1] if 1 <= pos <= len(seq) else "—"

# چاپ خلاصه
print(f"\n🔎 طول توالی: {len(seq)} aa")
print(f"🔢 پوزیشن‌ها (۱-بیسی): {pos_list}\n")

# جدول سادهٔ نتایج
header = ["Pos", "AA_at_Pos", "Expected", "Match?"]
rows = []
for i, pos in enumerate(pos_list):
    aa = aa_at(seq, pos)
    exp = exp_list[i] if i < len(exp_list) else ""
    ok = (aa == exp) if exp else ""
    rows.append([pos, aa, exp, "✅" if ok is True else ("❌" if ok is False else "")])

# چاپ مرتب
w = [max(len(str(x)) for x in col) for col in zip(header, *rows)] if rows else [5,10,9,7]
fmt = "  ".join("{:<" + str(k) + "}" for k in w)
print(fmt.format(*header))
print("-" * (sum(w) + 2*(len(w)-1)))
for r in rows:
    print(fmt.format(*r))

# نمای هایلایت خطی اطراف هر پوزیشن (±10 aa)
def window(seq: str, pos: int, half: int = 10) -> str:
    i = pos - 1
    start = max(0, i - half)
    end = min(len(seq), i + half + 1)
    frag = list(seq[start:end])
    if 0 <= i - start < len(frag):
        frag[i - start] = f"[{frag[i - start]}]"  # هایلایت
    return f"{start+1:>4}-{end:>4}: " + "".join(frag)

if pos_list:
    print("\n👁️  نماهای اطراف پوزیشن‌ها:")
    for p in pos_list:
        print(window(seq, p, half=10))



🔎 طول توالی: 150 aa
🔢 پوزیشن‌ها (۱-بیسی): [47, 51, 101]

Pos  AA_at_Pos  Expected  Match?
--------------------------------
47   S          S         ✅     
51   H          H         ✅     
101  D          D         ✅     

👁️  نماهای اطراف پوزیشن‌ها:
  37-  57: AGDAQVAVAG[S]QAGHRGLLAA
  41-  61: QVAVAGSQAG[H]RGLLAALALG
  91- 111: AAALLAALER[D]YAAFAALTPE
